# --- Day 24: Electromagnetic Moat ---

The CPU itself is a large, black building surrounded by a bottomless pit. Enormous metal tubes extend outward from the side of the building at regular intervals and descend down into the void. There's no way to cross, but you need to get inside.

No way, of course, other than building a bridge out of the magnetic components strewn about nearby.

Each component has two ports, one on each end. The ports come in all different types, and only matching types can be connected. You take an inventory of the components by their port types (your puzzle input). Each port is identified by the number of pins it uses; more pins mean a stronger connection for your bridge. A 3/7 component, for example, has a type-3 port on one side, and a type-7 port on the other.

Your side of the pit is metallic; a perfect surface to connect a magnetic, zero-pin port. Because of this, the first port you use must be of type 0. It doesn't matter what type of port you end with; your goal is just to make the bridge as strong as possible.

The strength of a bridge is the sum of the port types in each component. For example, if your bridge is made of components 0/3, 3/7, and 7/4, your bridge has a strength of 0+3 + 3+7 + 7+4 = 24.

For example, suppose you had the following components:

```
0/2
2/2
2/3
3/4
3/5
0/1
10/1
9/10
```

With them, you could make the following valid bridges:

```
0/1
0/1--10/1
0/1--10/1--9/10
0/2
0/2--2/3
0/2--2/3--3/4
0/2--2/3--3/5
0/2--2/2
0/2--2/2--2/3
0/2--2/2--2/3--3/4
0/2--2/2--2/3--3/5
```

(Note how, as shown by 10/1, order of ports within a component doesn't matter. However, you may only use each port on a component once.)

Of these bridges, the strongest one is `0/1--10/1--9/10`; it has a strength of `0+1 + 1+10 + 10+9 = 31`.

**What is the strength of the strongest bridge you can make with the components you have available?**

First up, converting the stringy input into ints stored in a list:

In [15]:
# the puzzle input
with open('puzzle_inputs/day24_input.txt') as f:
    data = f.read().strip().split("\n")
puzzle_input = [[int(i) for i in line.split("/")] for line in data]
print("Number of bridgers", len(puzzle_input))
puzzle_input[:6]

Number of bridgers 54


[[48, 5], [25, 10], [35, 49], [34, 41], [35, 35], [47, 35]]

Starting with the test input:

In [23]:
test_input = [[int(i) for i in l.split("/")] for l in """0/2
2/2
2/3
3/4
3/5
0/1
10/1
9/10""".strip().split("\n")]
test_input

[[0, 2], [2, 2], [2, 3], [3, 4], [3, 5], [0, 1], [10, 1], [9, 10]]

hmmm, this could be a graph problem, or recursive or even a brute force one. First up, some helper functions:

In [78]:
def is_bridge(bridge):
    """return true if bridge is valid, false otherwise"""
    size = len(bridge)
    if size < 1 : return False
    if bridge[0][0] != 0: return False
    
    for i in range(size-1):
        if bridge[i][1] != bridge[i+1][0]:
            return False
    return True

is_bridge([[0, 2], [2,1], [1,4]]), is_bridge(test_input)

(True, False)

In [89]:
from collections import defaultdict
def make_ports_dict(bridges=test_input):
    ports = defaultdict(list)
    for br in test_input:
        ports[br[0]].append(br)
    return ports
make_ports_dict()

defaultdict(list,
            {0: [[0, 2], [0, 1]],
             2: [[2, 2], [2, 3]],
             3: [[3, 4], [3, 5]],
             9: [[9, 10]],
             10: [[10, 1]]})

In [81]:
def next_valid_bridges(bridge, bridges):
    return [br for br in bridges if br[0] == bridge[1]]
next_valid_bridges([0,2], test_input)

[[2, 2], [2, 3]]

now to make the bridges. Since we start with zero:

In [80]:
start_bridges = [br for br in test_input if br[0]==0]
start_bridges

[[0, 2], [0, 1]]

In [84]:
graph = dict()